In [2]:
import assets.helper as b3
import assets.functions as run


In [3]:
import pandas as pd
import numpy as np
import os

import assets.graphs as g
import random
import requests

### QUOTES PRICES

In [ ]:
fund = run.load_pkl(f'{b3.app_folder}/fund')
quotes = run.load_pkl(f'{b3.app_folder}/quotes')


In [ ]:
fund.keys(), fund['BENS INDUSTRIAIS'].columns

In [ ]:
quotes.keys(), quotes['AERIS'].keys()

In [ ]:
df_preplot = {}
start_time = run.time.time()

for i, (setor, df_fund) in enumerate(fund.items()):
    df_fund = preprocess_data(df_fund)
    df_cd_conta = pivot_data(df_fund)

    df_unique = df_fund.drop_duplicates(subset=['DT_REFER', 'PREGAO']).drop(['CD_CONTA', 'DS_CONTA', 'VL_CONTA'], axis=1)
    df_premerged = pd.merge(df_unique, df_cd_conta, on=['DT_REFER', 'PREGAO'])
    df_resampled = resample_data(df_premerged)
    
    df_merged = merge_with_bigdata(df_resampled, bigdata)
    df_merged = df_merged.set_index('Date', drop=True)
    df_merged = df_merged.groupby('PREGAO', group_keys=False).apply(add_metrics)

    # Clean up the dataframe using the cleanup_dataframe function
    df_merged = cleanup_dataframe(df_merged)

    df_merged = add_metrics(df_merged)

    df_preplot[setor] = df_merged

    print(setor, run.remaining_time(start_time, len(fund), i))


In [ ]:
# Define the path to the folder
company_folder = 'company'
folder_path = os.path.join(b3.app_folder, company_folder)

# Check if the folder does not exist
if not os.path.exists(folder_path):
    # Create the folder
    os.makedirs(folder_path)


In [ ]:
start_time = run.time.time()
for i, (setor, df) in enumerate(df_preplot.items()):
    companies = df['PREGAO'].unique()
    for i2, company in enumerate(companies):
        mask = df['PREGAO'] == company
        df_temp = df[mask]
        try:
            df_temp = run.save_pkl(df_temp, f'{b3.app_folder}/{company_folder}/{company}')
        except Exception as e:
            pass
    print(setor, run.remaining_time(start_time, len(df_preplot), i))


In [ ]:
mask = df_preplot['BENS INDUSTRIAIS']['TICKER'] == 'AERI3'
df_preplot['BENS INDUSTRIAIS'][mask]

### DASH

In [ ]:
company = 'AZEVEDO'
df = run.load_pkl(f'{b3.app_folder}/{b3.company_folder}/{company}')
tickers = df['TICKER'].unique()
# df.columns.to_list()

In [ ]:
df['03.11 - Lucro Líquido']
df['14.04.01 - ROE (Resultado por Patrimônio)']
df['03.01 - Receita Bruta']
df['06.01 - Caixa das Operações']





In [ ]:
graphs = g.construct_graphs(df)
graphs.keys()

In [ ]:
import assets.lines
lines = assets.lines.report
groups = set(entry['title'][:2] for entry in lines)
groups

In [ ]:
for g, (group_key, group) in enumerate(graphs.items()):
    status = True if g == 0 else False
    plots = []
    for l, (line_key, line) in enumerate(group.items()):
        for p, (plot_key, plot_info) in enumerate(line.items()):
            for ticker in df['TICKER'].unique():
                df_ticker = df[df['TICKER'] == ticker]
                if not df.empty:
                    print(group[0][0]['info']['title'])
                    print(line[0]['info']['title'])
                    print(plot_info['info']['title'])
                    run.plot_tweak(plot_info, df_ticker).show();

In [ ]:
for ticker in tickers:
    df = df[df['TICKER'] == ticker]
    print(ticker, len(df))

    if not df.empty:
        for g, (group_key, group) in enumerate(merged_entries.items()):
            status = True if g == 0 else False
            plots = []
            for l, (line_key, line) in enumerate(group.items()):
                for p, (plot_key, plot_info) in enumerate(line.items()):
                    print(ticker)
                    run.plot_tweak(plot_info, df).show();

In [ ]:
fund = run.load_database()


In [ ]:
setores = []
for setor, df in fund.items():
    columns=['SETOR', 'SUBSETOR', 'SEGMENTO', 'PREGAO']
    setores.append(df[columns].drop_duplicates())
    print(setor)
sss = pd.concat(setores)
sss = sss.applymap(run.clean_text)

sss = run.save_pkl(sss, f'{b3.app_folder}/sss')

In [ ]:
sss

In [ ]:
mask = sss['SUBSETOR'] == 'MATERIAL DE TRANSPORTE'
sss[mask]

#### Playground


In [4]:
company_list = run.load_pkl('company_list')
company_list

FileNotFoundError: [Errno 2] No such file or directory: 'company_list.pkl'

In [ ]:
listagem_siglas = ['NM', 'N1', 'N2', 'MA', 'M2', 'MB', 'DR1', 'DR2', 'DR3', 'DRE', 'DRN']
listagem_extenso = ['Novo Mercado', 'Nível 1 de Governança Corporativa', 'Nível 2 de Governança Corporativa', 'Bovespa Mais', 'Bovespa Mais Nível 2', 'Organização Tradicional OTC', 'BDR Nível 1', 'BDR Nível 2', 'BDR Nível 3', 'BDR ETF', 'BDR Não Patrocinado']
list_dict = dict(zip(listagem_siglas, listagem_extenso))
list_dict

#### Merge and update


In [5]:
driver, wait = run.load_browser()
url = 'https://sistemaswebb3-listados.b3.com.br/listedCompaniesPage/search?language=pt-br' 



In [6]:
b3_cols = b3.cols_b3_companies + b3.col_b3_companies_extra_columns
companies_from_file = run.read_or_create_dataframe('company', b3_cols).fillna('')
b3_companies_tickers = run.load_pkl('b3_companies_tickers')

b3_companies_tickers = run.get_b3_tickers(driver, wait, url)
for col in b3_cols:
    if col not in b3_companies_tickers.columns:
        b3_companies_tickers[col] = ''
b3_companies_tickers = b3_companies_tickers[b3_cols]
# b3_companies_tickers = save_pkl(b3_companies_tickers, 'b3_companies_tickers')

key_columns = ['ticker', 'company_name']
len(b3_companies_tickers), len(companies_from_file)

company: total 120 items
found 2580 companies in 22 pages
4.55% 1+21, 0.641801s per item, Remaining: 0h 00m 13s page 1
9.09% 2+20, 0.626654s per item, Remaining: 0h 00m 12s page 2
13.64% 3+19, 0.621038s per item, Remaining: 0h 00m 11s page 3
18.18% 4+18, 0.621680s per item, Remaining: 0h 00m 11s page 4
22.73% 5+17, 0.620278s per item, Remaining: 0h 00m 10s page 5
27.27% 6+16, 0.619085s per item, Remaining: 0h 00m 09s page 6
31.82% 7+15, 0.617822s per item, Remaining: 0h 00m 09s page 7
36.36% 8+14, 0.616468s per item, Remaining: 0h 00m 08s page 8
40.91% 9+13, 0.617668s per item, Remaining: 0h 00m 08s page 9
45.45% 10+12, 0.616546s per item, Remaining: 0h 00m 07s page 10
50.00% 11+11, 0.616450s per item, Remaining: 0h 00m 06s page 11
54.55% 12+10, 0.615776s per item, Remaining: 0h 00m 06s page 12
59.09% 13+9, 0.616622s per item, Remaining: 0h 00m 05s page 13
63.64% 14+8, 0.616164s per item, Remaining: 0h 00m 04s page 14
68.18% 15+7, 0.616061s per item, Remaining: 0h 00m 04s page 15
72.73

(2580, 120)

In [21]:
merged = pd.merge(companies_from_file, b3_companies_tickers[:], how='outer', on=key_columns, indicator=True)

update_strict = merged[merged['_merge'] == 'right_only'][key_columns] # only companies new in web
update_broad = merged[merged['_merge'] != 'left_only'][key_columns] # all companies from web
updated = update_strict

len(update_broad), len(update_strict)

(2580, 2460)

In [22]:
key_columns = ['company_name', 'ticker']
b3_cols = b3.cols_b3_companies + b3.col_b3_companies_extra_columns
b3_cols_float = ['Capital Social']
b3_str_cols = [col for col in b3_cols if col not in  b3_cols_float]
col_types = {col: 'float' if col in b3_cols_float else 'str' for col in b3_cols}


In [ ]:
driver.get(b3.url)
# time.sleep(1)

size = len(updated)
# Iterate through the new companies to extract detailed information
new_companies = []
start_time = run.time.time()
for i, (index, row) in enumerate(updated.iterrows()):
    driver.get(b3.url)
    new_company = run.get_full_company_info(row, driver, wait)
    new_company = pd.DataFrame([new_company], columns=b3_cols).astype(col_types)
    new_company['Capital Social'] = new_company['Capital Social'].replace(np.nan, 0.0)
    new_company = new_company.replace('nan', '')
    new_company['Capital Social'] = pd.to_numeric(new_company['Capital Social'], errors='coerce').astype('float')

    try:
        if not new_company.empty:
            cnpj = new_company['cnpj'][0]
            url = f'https://cnpj.biz/{cnpj}'
            response = requests.get(url, headers={"User-Agent": random.choice(b3.USER_AGENTS)})
            extra = run.get_cnpj_info(response)
            extra = pd.DataFrame([extra], columns=b3_cols).fillna('')
            extra = extra.astype(col_types)
        else:
            extra = pd.DataFrame([], columns=b3_cols)

    except Exception as e:
        pass
    extra = extra.reset_index(drop=True)
    new_company = pd.merge(new_company[b3.cols_b3_companies], extra[b3.col_b3_companies_extra_columns], left_on='cnpj', right_on='CNPJ', how='outer').fillna('').reset_index(drop=True)
    new_companies.append(new_company)
    
    print(run.remaining_time(start_time, size, i), row['ticker'], row['company_name'])

    if (size-i-1) % (b3.bin_size/10) == 0:
        temp = pd.concat(new_companies).reset_index(drop=True)
        temp['Capital Social'] = pd.to_numeric(temp['Capital Social'], errors='coerce').astype('float')
        temp['Capital Social'] = temp['Capital Social'].replace(np.nan, 0.0)
        temp = pd.merge(companies_from_file, temp, on=b3_cols, how='outer', indicator=False).fillna('').drop_duplicates(subset=key_columns, keep='last').reset_index(drop=True)
        temp = save_and_pickle(temp, 'company')


In [132]:
temp

,pregao,company_name,cvm,listagem,ticker,tickers,asin,cnpj,site,setor,...,Opção pelo MEI,Opção pelo Simples,Capital Social,Tipo,Situação,Logradouro,Bairro,CEP,Município,Estado
0,,1461 INVESTIMENTOS SA,,,UQMU,,,,,NENHUM,...,,,0.000000e+00,,,,,,,
1,,3R PETROLEUM OLEO E GAS SA,,,RRRP,RRRP3,,12.091.809/0001-55,,PETROLEO GAS E BIOCOMBUSTIVEIS,...,Não,Não,5.054406e+09,Matriz,Ativa,"Praia Botafogo, 186",Botafogo,22250-145,Rio de Janeiro,Rio de Janeiro
2,,3R POTIGUAR SA,,,PTGU,,,,,NENHUM,...,,,0.000000e+00,,,,,,,
3,,3Z REALTY DESENVOLVIMENTO IMOBILIARIO SA,,,TZRD,,,08.145.855/0001-12,,NENHUM,...,Não,Não,8.968293e+08,Matriz,Ativa,"Avenida Jose de Souza Campos, 753",Cambui,13025-320,Campinas,São Paulo
4,,524 PARTICIPACOES SA,,,QVQP,QVQP3,,01.851.771/0001-55,http://www.524participacoes.com.br/home/index....,OUTROS,...,Não,Não,1.135905e+06,Matriz,Ativa,"Avenida Presidente Antonio Carlos, 00051",Centro,20020-010,Rio de Janeiro,Rio de Janeiro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,,AMPLA ENERGIA E SERVICOS SA,,,CBEE,CBEE3,,33.050.071/0001-58,http://www.eneldistribuicao.com.br/rj,UTILIDADE PUBLICA,...,Não,Não,4.438230e+09,Matriz,Ativa,"Avenida Oscar Niemeyer, 02000",Santo Cristo,20220-297,Rio de Janeiro,Rio de Janeiro
122,,ANDALI AS,,,ANDA,None,,02.227.264/0001-08,,NENHUM,...,,,0.000000e+00,,,,,,,
123,,,,,,,,,,,...,,,0.000000e+00,,,,,,,
124,,ANDORINHA ENERGIAS RENOVAVEIS SA,,,ANDH,None,,17.613.512/0001-26,,NENHUM,...,Não,Não,4.161815e+07,Matriz,Ativa,"Avenida Candido de Abreu, 70",Centro Civico,80530-000,Curitiba,Paraná


In [21]:
for col in b3_cols:
    d1 = new_company[col].dtype
    d2 = extra[col].dtype
    print(col, d1, d2)
    if d1 != d2:
        print(col, d1, d2)
    

pregao object object
company_name object object
cvm object object
listagem object object
ticker object object
tickers object object
asin object object
cnpj object object
site object object
setor object object
subsetor object object
segmento object object
atividade object object
escriturador object object
url object object
CNPJ object object
Razão Social object object
Nome Fantasia object object
Data da Abertura object object
Porte object object
Natureza Jurídica object object
Opção pelo MEI object object
Opção pelo Simples object object
Capital Social float64 float64
Tipo object object
Situação object object
Logradouro object object
Bairro object object
CEP object object
Município object object
Estado object object


In [22]:
b3_cols

['pregao',
 'company_name',
 'cvm',
 'listagem',
 'ticker',
 'tickers',
 'asin',
 'cnpj',
 'site',
 'setor',
 'subsetor',
 'segmento',
 'atividade',
 'escriturador',
 'url',
 'CNPJ',
 'Razão Social',
 'Nome Fantasia',
 'Data da Abertura',
 'Porte',
 'Natureza Jurídica',
 'Opção pelo MEI',
 'Opção pelo Simples',
 'Capital Social',
 'Tipo',
 'Situação',
 'Logradouro',
 'Bairro',
 'CEP',
 'Município',
 'Estado']

##### CVM GET CODE

In [ ]:
for i in range(0,1000+1):
    url = f'https://sistemaswebb3-listados.b3.com.br/listedCompaniesPage/main/{i}/ABCD/overview?language=pt-br'
    print(url)